In [ ]:
# importamos las librerias necesarias
import pandas as pd
import faker as fk
import random as rd
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import openpyxl
import sympy as sp

In [ ]:
planetas = {
    "Tierra": 9.8,
    "Marte": 3.7,
    "Júpiter": 24.8,
    "Saturno": 9.0,
    "Urano": 8.7,
    "Neptuno": 11.0
}

In [ ]:
# Para el apartado A
L1 = 1.00
t1 = 0.46
g_a = 2*L1 / (t1**2)
print("La aceleración de la gravedad es: ", g_a)

In [ ]:
# Para el apartado B
L2 = 2.00
t2 = 0.66
g_b = 2*L2 / (t2**2)
print("La aceleración de la gravedad es: ", g_b)

In [ ]:
# creamos un dataframe con los datos
df = pd.DataFrame({
    'Longitud': [L1, L2],
    'Tiempo': [t1, t2],
    'Gravedad': [g_a, g_b],
    'Planeta más cercano': ['Tierra', 'Saturno'],
    '2ndo más cercano': ['Saturno', 'Tierra']},index= ['Escenario A', 'Escenario B'])
df

In [ ]:
def calcular_error(L, t):
    L_simbolo, t_simbolo = sp.symbols('L t')
    g_calculada_simbolo = (2*L_simbolo) / (t_simbolo**2)
    
    derivada_parcial_L = sp.diff(g_calculada_simbolo, L_simbolo)
    derivada_parcial_t = sp.diff(g_calculada_simbolo, t_simbolo)
    
    error_L = 0.01
    error_t = 0.01
    
    error_g_calculada = sp.sqrt((derivada_parcial_L * error_L)**2 + (derivada_parcial_t * error_t)**2)
    
    return float(error_g_calculada.subs({L_simbolo: L, t_simbolo: t}))

    

In [ ]:
fake = fk.Faker()

def generar_datos(planetas, L_objetivo, t_objetivo):
    L = round(rd.uniform(L_objetivo - 0.1, L_objetivo + 0.1), 2)
    t = round(rd.uniform(t_objetivo - 0.1, t_objetivo + 0.1), 2)
    g_calculada = round((2*L) / (t**2), 2)

    diferencia_minima = float('inf')
    planeta_mas_cercano = ''
    
    diferencia_minima_2 = float('inf')
    segundo_planeta_mas_cercano = ''

    for planeta, gravedad in planetas.items():
        diferencia = abs(gravedad - g_calculada)
        if diferencia < diferencia_minima:
            diferencia_minima = diferencia
            planeta_mas_cercano = planeta
    
    for planeta, gravedad in planetas.items():
        if planeta != planeta_mas_cercano:
            diferencia = abs(gravedad - g_calculada)
            if diferencia < diferencia_minima_2:
                diferencia_minima_2 = diferencia
                segundo_planeta_mas_cercano = planeta
    
    error_g_calculada = calcular_error(L, t)
    error_porcentaje = (error_g_calculada / g_calculada) * 100
    
    return {
        'Id': fake.uuid4(),
        'Longitud (m)': L,
        'Tiempo (s)': t,
        'Gravedad (m/s^2)': g_calculada,
        'Planeta más cercano': planeta_mas_cercano,
        'Segundo Planeta más cercano': segundo_planeta_mas_cercano,
        'Error Gravedad (m/s^2)': error_g_calculada,
        'Error Porcentaje (%)': error_porcentaje
    }

df_a = pd.DataFrame([generar_datos(planetas, L1, t1) for _ in range(1000)])
df_a



In [ ]:
df_b = pd.DataFrame([generar_datos(planetas, L2, t2) for _ in range(1000)])
df_b

In [ ]:
def analizar_dataframe(df):
    # Estadísticas descriptivas
    print(df.describe())
    # Histogramas
    df.hist(column=['Longitud (m)', 'Tiempo (s)', 'Gravedad (m/s^2)'])
    plt.show()
    # Gráfico de dispersión
    plt.scatter(df['Longitud (m)'], df['Tiempo (s)'])
    plt.xlabel('Longitud (m)')
    plt.ylabel('Tiempo (s)')
    plt.show()
    # codificar etiquetas de planetas
    codificador_etiquetas = LabelEncoder()
    df['Planeta más cercano (codigo)'] = codificador_etiquetas.fit_transform(df['Planeta más cercano'])
    df['Segundo Planeta más cercano (codigo)'] = codificador_etiquetas.fit_transform(df['Segundo Planeta más cercano'])
    # correlación entre variables
    df_numeric = df.select_dtypes(include=[np.number])
    print(df_numeric.corr())
    # regresión lineal
    X = df['Longitud (m)'].values.reshape(-1, 1)
    y = df['Tiempo (s)'].values.reshape(-1, 1)

    regresion_lineal = LinearRegression()
    regresion_lineal.fit(X, y)

    y_pred = regresion_lineal.predict(X)
    plt.scatter(X, y, color='blue') 
    plt.plot(X, y_pred, color='red')
    plt.xlabel('Longitud (m)')
    plt.ylabel('Tiempo (s)')
    plt.show()


def contar_planetas(dataframe):
    conteo_planetas = dataframe['Planeta más cercano'].value_counts()
    
    planeta_mas_cercana_nave = conteo_planetas.idxmax()
    
    conteo_planetas_2 = dataframe['Segundo Planeta más cercano'].value_counts()
    
    segundo_planeta_mas_cercana_nave = conteo_planetas_2.idxmax()
    
    return conteo_planetas, planeta_mas_cercana_nave, conteo_planetas_2, segundo_planeta_mas_cercana_nave

In [ ]:
analizar_dataframe(df_a)
conteo_planetas_a, planeta_mas_cercana_nave_a, conteo_planetas_2_a, segundo_planeta_mas_cercana_nave_a = contar_planetas(df_a)

print(conteo_planetas_a)
print(f'El planeta más cercano a la nave es: {planeta_mas_cercana_nave_a}')
print(f'El segundo planeta más cercano a la nave es: {segundo_planeta_mas_cercana_nave_a}')

In [ ]:
analizar_dataframe(df_b)
conteo_planetas_b, planeta_mas_cercana_nave_b, conteo_planetas_2_b, segundo_planeta_mas_cercana_nave_b = contar_planetas(df_b)

print(conteo_planetas_b)
print(f'El planeta más cercano a la nave es: {planeta_mas_cercana_nave_b}')
print(f'El segundo planeta más cercano a la nave es: {segundo_planeta_mas_cercana_nave_b}')

In [ ]:
dataframe_combinado = pd.concat([df_a, df_b], ignore_index=True)

analizar_dataframe(dataframe_combinado)
conteo_planetas_combinado, planeta_mas_cercana_nave_combinado, conteo_planetas_2_combinado, segundo_planeta_mas_cercana_nave_combinado = contar_planetas(dataframe_combinado)

print(conteo_planetas_combinado)
print(f'El planeta más cercano a la nave es: {planeta_mas_cercana_nave_combinado}')
print(f'El segundo planeta más cercano a la nave es: {segundo_planeta_mas_cercana_nave_combinado}')

In [ ]:
# creamos un archivo excel con los datos
def crear_excel_exp_A():
    df = pd.DataFrame(df_a)
    df.to_excel('Datos_experimentos_A.xlsx', index=False)
    print('Archivo A creado con éxito')


def crear_excel_exp_B():
    df = pd.DataFrame(df_b)
    df.to_excel('Datos_experimentos_B.xlsx', index=False)
    print('Archivo  B creado con éxito')

crear_excel_exp_A()
crear_excel_exp_B()